<a href="https://colab.research.google.com/github/soohyunme/TensorFlow_Tutorial/blob/main/Code/03_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from threading import active_count

from tensorflow.python.keras.layers.core import Activation
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10


# Device setting

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

# Load data

In [ ]:
(x_train,y_train), (x_test,y_test) = cifar10.load_data()

# Normalize

In [ ]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Initialize model

In [ ]:
model = keras.Sequential(
    [
     keras.Input(shape=(32,32,3)),
     layers.Conv2D(32, 3, padding='valid', activation='relu'),
     layers.MaxPooling2D(pool_size=(2,2)),
     layers.Conv2D(64, 3, activation='relu'),
     layers.MaxPooling2D(pool_size=(2,2)),
     layers.Conv2D(128, 3, activation='relu'),
     layers.Flatten(),
     layers.Dense(64,activation='relu'),
     layers.Dense(10),

    ]
)

In [ ]:
print(model.summary())

In [ ]:
def my_model():
  with tf.device('/device:GPU:0'):
      inputs = keras.Input(shape=(32, 32, 3))
      x = layers.Conv2D(32, 3)(inputs)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      x = layers.MaxPooling2D()(x)
      
      x = layers.Conv2D(64, 3)(x)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      x = layers.MaxPooling2D()(x)
      
      x = layers.Conv2D(128, 3)(x)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      
      x = layers.Flatten()(x)
      x = layers.Dense(64, activation="relu")(x)
      outputs = layers.Dense(10)(x)
      model = keras.Model(inputs=inputs, outputs=outputs)
      return model

## New model


In [ ]:
def new_model():
   with tf.device('/device:GPU:0'):
      inputs = keras.Input(shape=(32,32,3))
      x = layers.Conv2D(32,3)(inputs)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      x = layers.MaxPooling2D()(x)
      x = layers.Dropout(0.2)(x)
      
      x = layers.Conv2D(64, 3, padding='same')(x)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      x = layers.Dropout(0.2)(x)

      x = layers.Conv2D(128,3)(x)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      x = layers.Dropout(0.2)(x)
      
      x = layers.Flatten()(x)
      x = layers.Dense(64, activation='relu')(x)
      outputs = layers.Dense(10)(x)
      model = keras.Model(inputs=inputs, outputs=outputs)
      return model


In [ ]:
# model = my_model()
model = new_model()

In [ ]:
print(model.summary())


# Model compile

In [ ]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(lr=3e-4),
    metrics=['accuracy'],
)

# Train and evaluate

In [ ]:
with tf.device('/device:GPU:0'):
  history = model.fit(x_train, y_train, batch_size=64, epochs=30, verbose=2)
  model.evaluate(x_test, y_test, batch_size=64, verbose=2)

# SUGGESTIONS


1. **What accuracy can you get on the test set by training longer, increasing the model size, changing kernel sizes, etc?**
> Baseline : 0.7197  
> Add MaxPooling each Conv2D : 0.6812  
> More Conv2D layer : 0.7133  
> Change kernel size : 0.6547  
> Change kernel size +  more epochs : 0.6669  
> Increase batch size + more epochs : 0.7007  
> Increase batch size : 0.6738  
> (Baseline) Apply drop out(0.2) + more epochs(30) : 0.7732

2. **In the last video we train a FC on MNIST: what can you get by using a conv net instead?**
> FC보다 파라미터의 수를 줄일 수 있음  
> 이미지의 형상을 고려하여 학습할 수 있음  
> 보다 적은 양의 학습 데이터로도 더 빠른 시간으로 학습이 가능